<a href="https://colab.research.google.com/github/buzo14/titanic-survivor-prediction/blob/main/Titanic_survivor_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"henryochiabuto","key":"ce8af37b0718cba3f3046a0b321bc303"}'}

In [3]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
#!kaggle datasets list

In [4]:
!kaggle competitions download -c titanic

  0% 0.00/34.1k [00:00<?, ?B/s]
100% 34.1k/34.1k [00:00<00:00, 52.1MB/s]


In [6]:
import zipfile
zip_ref = zipfile.ZipFile('/content/titanic.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
# Importing pandas for analysis
import pandas as pd

In [7]:
# Using pandas to read in our csv files
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')

In [8]:
# Three columns are dropped that have little or no use to our model

train_data = train_df.drop(['Name', 'Ticket', 'Cabin'], axis=1)
test_data = test_df.drop(['Name', 'Ticket', 'Cabin'], axis=1)

print(train_data.head())
print(test_data.head())

   PassengerId  Survived  Pclass     Sex   Age  SibSp  Parch     Fare Embarked
0            1         0       3    male  22.0      1      0   7.2500        S
1            2         1       1  female  38.0      1      0  71.2833        C
2            3         1       3  female  26.0      0      0   7.9250        S
3            4         1       1  female  35.0      1      0  53.1000        S
4            5         0       3    male  35.0      0      0   8.0500        S
   PassengerId  Pclass     Sex   Age  SibSp  Parch     Fare Embarked
0          892       3    male  34.5      0      0   7.8292        Q
1          893       3  female  47.0      1      0   7.0000        S
2          894       2    male  62.0      0      0   9.6875        Q
3          895       3    male  27.0      0      0   8.6625        S
4          896       3  female  22.0      1      1  12.2875        S


In [9]:
print(train_data.info())
print(test_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    object 
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Fare         891 non-null    float64
 8   Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(2)
memory usage: 62.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Sex          418 non-null    object 
 3   Age          332 non-null    float64
 4   SibSp

In [10]:
# Our feature columns are divided into numerical and categorical lists
num_cols = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
cat_cols = ['Sex', 'Embarked']

X = train_data[num_cols+cat_cols]
y = train_data['Survived']

In [11]:
# Train test split is imported and used to split our dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [12]:
# Preprocessing packages and Pipeline are imported, a pipeline is used to preprocess our dataset
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline

num_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='mean')),
    ('scale', MinMaxScaler())
])
cat_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('one-hot', OneHotEncoder())
])

In [13]:
# ColumnTransformer is imported
from sklearn.compose import ColumnTransformer

col_trans = ColumnTransformer(transformers=[
    ('num_pipeline',num_pipeline,num_cols),
    ('cat_pipeline',cat_pipeline,cat_cols)
    ],
    remainder='drop',
    n_jobs=-1)

In [14]:
# Using LogisticRegression, we pass in our model into a pipeline
from sklearn.linear_model import LogisticRegression

lg = LogisticRegression(max_iter=10000, random_state=14)
lg_pipeline = Pipeline(steps=[
    ('col_trans', col_trans),
    ('model', lg)
])

In [15]:
# We fit the data into our model and calculate its accuracy

from sklearn.metrics import confusion_matrix, accuracy_score
lg_pipeline.fit(X_train, y_train)
preds = lg_pipeline.predict(X_test)
cm_lg = confusion_matrix(y_test,preds)
accuracy_lg = accuracy_score(y_test,preds)
print(cm_lg, accuracy_lg)

[[96 19]
 [20 44]] 0.7821229050279329


In [16]:
#  Using SVM, we pass in our model into a pipeline

from sklearn.svm import SVC
svm = SVC(kernel = 'linear', random_state=14)

svm_pipeline = Pipeline(steps=[
    ('col_trans', col_trans),
    ('model', svm)
])
svm_pipeline.fit(X_train, y_train)
pred_svm = svm_pipeline.predict(X_test)

cm_svm= confusion_matrix(y_test,pred_svm)
accuracy_svm = accuracy_score(y_test,pred_svm)

print(cm_svm)
print(accuracy_svm)

[[98 17]
 [23 41]]
0.776536312849162


In [17]:
Xt = test_data[num_cols+cat_cols]
preds = lg_pipeline.predict(Xt)

In [18]:
test_result = pd.DataFrame(test_data['PassengerId'])
test_result.insert(1, 'Survived', preds)
test_result.to_csv('test_result.csv', index=False)